# 1.Introduction

This Notebook tutorial will allow you to perform Public Key rotation securely on your TrustFLEX device.

## 1.1.Tutorial Prerequisites:
    

The code runs on python 3.x environment. Folowing python modules are needed to run the example

1. cryptoauthlib
2. cryptography
3. ipywidgets

The next step of the tutorial with attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

In [ ]:
!python -m pip install -r ../../../requirements.txt

## 1.2. Load the necessary modules and helper functions

This Tutorial uses a number of python helper functions that are loaded by the following cell.
The below code does the following things:
1. Setup of cryptography module
2. Add helper functions to get device name and device type

In [ ]:
from cryptoauthlib import *
from cryptoauthlib.iface import *
import os
from common import *
from public_key_rotation_helper import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import argparse,time
import binascii
from cryptoauthlib.device import *
from cryptoauthlib.library import ctypes_to_bytes

# 2. Tutorial

The CryptoAuthentication device supports updating the public key after it has been validated. The public keys that are intended to rotate securely should be stored in slots marked as PubInvalid. 

The slot validation and invalidation is the process in the which the authenticity of the slot will be verified using its digest and signature provided by validation authority. Once its the authenticated, slot can be marked as validated or invalidated.

Once the key is validated a new key cannot be written before it is invalidated.

Secure Public key rotation is a multi step process,
1. Establish the communication between host(PC) and TrustFLEX
2. Generating a Validation Authority Key pair
3. Rotate new public key
4. Validate and verify rotated public key is usability

## 2.1. Establish the communication between host (PC) and TrustFLEX
1. Initialize the library which also loads it
2. Check if the correct device is connected to the PC. If the wrong device is connected, a ValueError excpetion will be raised.

### 2.1.1. Select the CryptoAuthLib Hardware Abstraction layer to KIT USB HID

In [ ]:
ATCA_KIT_I2C_IFACE = 1
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6C #TFLXTLS device I2C address
ATCA_SUCCESS = 0
print("Set devtype as ECC608A - Successful")

### 2.1.2. Establish the connection with TrustFLEX device on the USB dongle and check its type
<font color=red>If the next Tutorial step yields an exception, the ECC608A is not correctly connected or of the wrong type.</font>

In [ ]:
# Initialize interface
assert atcab_init(cfg) == ATCA_SUCCESS, "Can't connect to the USB dongle"

# Get connected device type
info = bytearray(4)
assert atcab_info(info) == ATCA_SUCCESS, "Can't read the ECC608A device information"
dev_type = get_device_type_id(get_device_name(info))

# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    assert atcab_release() == ATCA_SUCCESS
    raise ValueError('Device not supported')
    
print("Device initialization - Successful")

## 2.2.Generating a Validation Authority key pair
The most important step is to have the Validation Authority ECC P256 key pair. The key pair can be generated using many of the options available like openssl (or) python modules etc..,. The private key should be securely stored by OEM and the corresponding public key should be written to TrustFLEX devices.

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\Crypto Resource Generator.ipynb. </font> This key pair will be used for this public key rotation. 

<font color = green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

<center><img src="img/1.png" alt="**Get your Secure Elements here!**" /></center>

### 2.3 Rotate new public key
The PubInvalid slots update is possible only after invalidating the slot i.e. only invalidated slots are allowed to write.

### 2.3.a Generating a new public key
Generate a new key pair for system to rotate the existing public key with this new one. The new pubic key will be authorized by the validation authority. It calculates the new public keys digest and signs the same.

<center><img src="img/2-3-a.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def generate_key_pair(b):
    generate_new_rotate_public_key_pair()
    rotating_key_pair_generate.button_style = 'success'
    print ('Generated new rotating public key pair\n')

rotating_key_pair_generate = widgets.Button(description = "Gen Rotating key pair", tooltip = 'Generate new rotating private/public key pair')
rotating_key_pair_generate.on_click(generate_key_pair)

### 2.3.b Authorize public key
To validate or invalidate, its required to have Public keys digest and signature provided by validation authority. This ensures the key is coming from the right source. Validation authority calculates the digest and signs it. Authority public key on the product verifies the same as part of validate/invalidate process.

<center><img src="img/2-3-b.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
nonce = os.urandom(32)
rotating_public_key_digest = bytearray(32)
signature = bytearray(64)

def Authority_key_validate_rotate_public_key(b):
    global signature
    global nonce
    
    print ('Signing the rotating key digest with the Authority Private Key\n')
    authority_private_key = get_validating_authority_key()
    rotating_private_key = get_rotating_key()
    rotating_public_key = rotating_private_key.public_key().public_bytes(encoding=Encoding.X962, format=PublicFormat.UncompressedPoint)[1:]
    

    rotating_public_key_digest = public_key_digest(nonce, rotating_public_key)
    signature = sign_host(rotating_public_key_digest, authority_private_key)
    Authorise.button_style = 'success'

        
    resource_generate()
        
Authorise = widgets.Button(description = "Authorize key", tooltip = 'Authorise the public key')
Authorise.on_click(Authority_key_validate_rotate_public_key)

### 2.3.c Update public key
If a Public key is already stored in the slot and is validated, unless its invalidated it is not possible to overwrite the slot. Once it is invalidated, the new pubic key can be written to slot.

### 2.3.d Validate the rotating public key
The new public key is updated after invalidating the existing key. Once the new key is written to the slot, its important to validate it. The successful validation restricts further writes to the slot until its invalidated. Also, after validating the slot can be used for cryptographic operations. 

The validation authority public key in the device verifies the signature to validate the slot.

<center><img src="img/2-3-d.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def rotate_public_key_write_and_validate(b):
    valid_buf = bytearray(4)
    global signature
    global nonce
    
    assert atcab_read_zone(ATCA_ZONE_DATA, rotating_key_slot, 0, 0, valid_buf, 4) == ATCA_SUCCESS

    if(valid_buf[0] >> 4 == 0x05):
        print ('Already Rotating Public key slot is validated,so invalidating the slot\n')
        assert public_key_invalidate() == 1
        print ('Invalidated the slot\n')

    rotating_private_key = get_rotating_key()
    rotating_public_key = rotating_private_key.public_key().public_bytes(encoding=Encoding.X962, format=PublicFormat.UncompressedPoint)[1:]
    assert atcab_write_pubkey(rotating_key_slot,rotating_public_key) == ATCA_SUCCESS
    print ('Rotating Public Key is written to device\n')

    assert public_key_validate_from_authority(nonce,signature) == 1
    
    print ('Rotating Public Key is validated with authority signed signature\n')
    write_and_validate_key.button_style = 'success'

write_and_validate_key = widgets.Button(description = "Validate key", tooltip = 'Write and validate rotating public key')
write_and_validate_key.on_click(rotate_public_key_write_and_validate)

## 2.4. Verify the rotating public key
This step verifies the rotating public written to device is validated and can be used for cryptographic operations.
The rotating private key signs a message on the host and is verified with the rotating public in slot

<center><img src="img/2-4.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def verify_rotate_public_key(b):
    is_verified_status = AtcaReference(False)

    message = os.urandom(32)
    print ('Signing with the Rotating Private Key\n')
    rotating_private_key = get_rotating_key()
    signature = sign_host(message,rotating_private_key)
    status = atcab_verify_stored(message, signature, rotating_key_slot, is_verified_status)

    if(is_verified_status == True):
        print("Verified the Rotating Public key\n")
        verify.button_style = 'success'

    else:
        print("Verify failed")
        verify.button_style = 'danger'
    public_key_invalidate()
        
verify = widgets.Button(description = "Verify key", tooltip = 'Verify key')
verify.on_click(verify_rotate_public_key)
display(widgets.HBox((rotating_key_pair_generate, Authorise, write_and_validate_key, verify)))